In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [10]:
lossess = pd.read_csv('./GE - Historical.csv', index_col=0, parse_dates=True)
# lossess = lossess.sort_index()
losses = lossess.Open.pct_change().dropna()
losses.head()

Date
2007-01-04    0.014969
2007-01-05   -0.010535
2007-01-08   -0.003194
2007-01-09    0.008812
2007-01-10   -0.010588
Name: Open, dtype: float64

In [11]:
# Import the Normal distribution and skewness test from scipy.stats
from scipy.stats import norm, anderson

# Fit portfolio losses to the Normal distribution
params = norm.fit(losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = norm.ppf(0.95, *params)
print("VaR_95, Normal distribution: ", VaR_95)

# Test the data for Normality
print("Anderson-Darling test result: ", anderson(losses))

VaR_95, Normal distribution:  0.050486968081522296
Anderson-Darling test result:  AndersonResult(statistic=27.07604542415197, critical_values=array([0.573, 0.653, 0.783, 0.913, 1.086]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=-0.0007058704271876911, scale=0.03114369508615258)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


In [12]:
# Import the skew-normal distribution and skewness test from scipy.stats
from scipy.stats import skewnorm, skewtest

# Test the data for skewness
print("Skewtest result: ", skewtest(losses))

# Fit the portfolio loss data to the skew-normal distribution
params = skewnorm.fit(losses)

# Compute the 95% VaR from the fitted distribution, using parameter estimates
VaR_95 = skewnorm.ppf(0.95, *params)
print("VaR_95 from skew-normal: ", VaR_95)

Skewtest result:  SkewtestResult(statistic=8.059974034733527, pvalue=7.631065621475823e-16)
VaR_95 from skew-normal:  0.051961463294927915


In [23]:
asset_returns = pd.read_csv('./crisis_portfolio.csv', index_col=0, parse_dates=True,date_format='%Y-%m-%d')
asset_returns = asset_returns.pct_change().dropna()
weights = [0.25, 0.25, 0.25, 0.25]

In [24]:
# Create portfolio returns for the two sub-periods using the list of asset returns
portfolio_returns = np.array([x.dot(weights) for x in asset_returns])

# Derive portfolio losses from portfolio returns
losses = - portfolio_returns

# Find the historical simulated VaR estimates
VaR_95 = [np.quantile(x, 0.95) for x in losses]

# Display the VaR estimates
print("VaR_95, 2005-2006: ", VaR_95[0], '; VaR_95, 2007-2009: ', VaR_95[1])

AttributeError: 'str' object has no attribute 'dot'

In [29]:
# Initialize daily cumulative loss for the assets, across N runs
N = 10000
daily_loss = np.zeros((4,N))
e_cov =  np.array([[0.00209328, 0.00114596, 0.00081893, 0.0010135 ],
       [0.00114596, 0.00192715, 0.00097157, 0.00082012],
       [0.00081893, 0.00097157, 0.00089666, 0.00064216],
       [0.0010135 , 0.00082012, 0.00064216, 0.00107184]])
total_steps = 1440
mu = np.array([[ 0.00048534],
       [-0.00042112],
       [-0.00074171],
       [-0.00056848]])
# Create the Monte Carlo simulations for N runs
for n in range(N):
    # Compute simulated path of length total_steps for correlated returns 
    correlated_randomness = e_cov @ norm.rvs(size = (4,total_steps))
    # Adjust simulated path by total_steps and mean of portfolio losses
    steps = 1/total_steps
    minute_losses = mu * steps + correlated_randomness * np.sqrt(steps)
    daily_loss[:, n] = minute_losses.sum(axis=1)
    
# Generate the 95% VaR estimate
losses = weights @ daily_loss
print("Monte Carlo VaR_95 estimate: ", np.quantile(losses, 0.95))

Monte Carlo VaR_95 estimate:  0.0031675254576541724


In [38]:
df = pd.read_csv('./crisis_portfolio.csv', index_col=0, parse_dates=True,date_format='%Y-%m-%d')
df = df.pct_change().dropna()
df

,Citibank,Morgan Stanley,Goldman Sachs,J.P. Morgan
Date,,,,
03/01/2005,0.001868,0.006844,0.008747,0.003589
04/01/2005,-0.008494,-0.010734,-0.006479,-0.018902
05/01/2005,0.012537,-0.005787,-0.004507,0.002083
06/01/2005,0.009699,0.023645,0.013776,0.005716
07/01/2005,-0.005722,-0.003909,-0.004276,-0.008008
...,...,...,...,...
23/12/2010,-0.010571,0.000365,-0.011792,-0.001897
27/12/2010,0.019231,0.003648,0.013305,0.014021
28/12/2010,0.002096,0.005453,-0.003768,-0.001406


In [40]:
df.Citibank.resample('Q').mean().dropna()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:
# Create a plot of quarterly minimum portfolio returns
plt.plot(port_q_min, label="Quarterly minimum return")

# Create a plot of quarterly mean volatility
plt.plot(vol_q_mean, label="Quarterly mean volatility")

# Create legend and plot
plt.legend()
plt.show()

In [ ]:
# Import the statsmodels API to be able to run regressions
import statsmodels.api as sm

# Add a constant to the regression
mort_del = sm.add_constant(mort_del)

# Regress quarterly minimum portfolio returns against mortgage delinquencies
result = sm.OLS(port_q_min, mort_del).fit()

# Retrieve the sum-of-squared residuals
ssr_total = result.ssr
print("Sum-of-squared residuals, 2005-2010: ", ssr_total)

In [41]:
# Add intercept constants to each sub-period 'before' and 'after'
before_with_intercept = sm.add_constant(before['mort_del'])
after_with_intercept  = sm.add_constant(after['mort_del'])

# Fit OLS regressions to each sub-period
r_b = sm.OLS(before['returns'], before_with_intercept).fit()
r_a = sm.OLS(after['returns'], after_with_intercept).fit()

# Get sum-of-squared residuals for both regressions
ssr_before = r_b.ssr
ssr_after = r_a.ssr
# Compute and display the Chow test statistic
numerator = ((ssr_total - (ssr_before + ssr_after)) / 2)
denominator = ((ssr_before + ssr_after) / (24 - 4))
print("Chow test statistic: ", numerator / denominator)

NameError: name 'sm' is not defined

In [ ]:
# Find the time series of returns with and without Citibank
ret_with_citi = prices_with_citi.pct_change().dot(weights_with_citi)
ret_without_citi = prices_without_citi.pct_change().dot(weights_without_citi)

# Find the average 30-day rolling window volatility as the standard deviation
vol_with_citi = ret_with_citi.rolling(30).std().dropna().rename("With Citi")
vol_without_citi = ret_without_citi.rolling(30).std().dropna().rename("Without Citi")

# Combine two volatilities into one Pandas DataFrame
vol = pd.concat([vol_with_citi, vol_without_citi], axis=1)

# Plot volatilities over time
vol.plot().set_ylabel("Losses")
plt.show()

In [ ]:
# Compute the 95% VaR on 2009-2010 losses
VaR_95 = np.quantile(estimate_data, 0.95)

# Find backtest_data exceeding the 95% VaR
extreme_values = backtest_data[backtest_data > VaR_95]

# Compare the fraction of extreme values for 2007-2008 to the Var_95 estimate
print("VaR_95: ", VaR_95, "; Backtest: ", len(extreme_values) / len(backtest_data) )

# Plot the extreme values and look for clustering
plt.stem(extreme_values.index, extreme_values)
plt.ylabel("Extreme values > VaR_95"); plt.xlabel("Date")
plt.show()